In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nitin\\DSML\\100.MLOPS\\10-MLOPS-Project-1\\experiment'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nitin\\DSML\\100.MLOPS\\10-MLOPS-Project-1'

In [5]:
import pandas as pd
data = pd.read_csv("artifact\data-ingestion\winequality-red.csv")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [6]:
data.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [7]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [8]:
data.shape

(1599, 12)

# DATA VALIDATION

In [9]:
### 📁 entity/ → 🗂️ ONLY DEFINE data entities (e.g., input/output dataclasses, structured schemas).
from dataclasses import dataclass
from pathlib import Path

@dataclass 
class DataValidation_config:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict

In [10]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml, create_directories
##📁 config/ -> ⚙️Python helpers to load/parse config.yaml and schema.yaml.

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        # self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifact_dir])

    
    def get_data_validation_config(self) -> DataValidation_config :
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidation_config(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.DataScience import logger

In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidation_config):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = True

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns) ##columns in data

            all_schema = self.config.all_schema.keys() ##columns according to us

            
            for col in all_cols:
                if col not in all_schema:  ## if data_columns not match with our_columns
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation status -> {col}: {validation_status}❗ \n")

            if validation_status:
                validation_status = True
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"✅ All columns are valid. Validation status True. \n")

            return validation_status
        
        except Exception as e:
            raise e
        
    def validate_columns_type(self)-> bool:
        try:
            data = pd.read_csv(self.config.unzip_data_dir)
            data_col_dtype = data.dtypes  # This returns a Series

            validation_status = True  # Assume everything is valid unless we find an error

            for col, expected_dtype in self.config.all_schema.items():
                actual_dtype = str(data_col_dtype[col])

                if actual_dtype != expected_dtype:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"❌ Data type mismatch for column '{col}': expected {expected_dtype}, got {actual_dtype}\n Validation status -> : {validation_status} ")
                    break  # You can remove break if you want to log all mismatches

            if validation_status:
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write("✅ All column data types are valid. Validation status True\n")

            return validation_status
        except Exception as e:
            raise e

In [13]:
%pwd


'c:\\Users\\nitin\\DSML\\100.MLOPS\\10-MLOPS-Project-1'

In [14]:

try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_validation_config()
    data_ingestion=DataValiadtion(config=data_ingestion_config)
    data_ingestion.validate_all_columns()
    data_ingestion.validate_columns_type()
except Exception as e:
    raise e

Trying to load YAML from: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\config\config.yaml
2025-09-30 21:21:31,908: INFO: src.DataScience: yaml file: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\config\config.yaml loaded successfully
Trying to load YAML from: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\schema.yaml
2025-09-30 21:21:31,920: INFO: src.DataScience: yaml file: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\schema.yaml loaded successfully
2025-09-30 21:21:31,923: INFO: src.DataScience: created directory at: artifact
2025-09-30 21:21:31,926: INFO: src.DataScience: created directory at: artifact/data_validation
